In [2]:
using Plots
using LaTeXStrings

# Modelo de Hodgkin-Huxley
El modelo de Hodgkin-Huxley fue propuesto en 1952 tras haber estudiado la fisiología del axón gigante del calamar y marcó un hito en historia de la neurociencia, a partir de ese momento muchos modelos se han desarrollado tomándolo como base. En su forma clásica el modelo se define como un sistema de cuatro equaciones diferenciales como se muestra a continuación:

\begin{align*}
 C_m \frac{dV}{dt} &= -g_{Na}m^3h(V-E_{Na}) - g_K n^4 (V - E_K) \\
                   &- g_L (V - E_L) + I_{ext} \\
 \frac{1}{\gamma(T)} \frac{dn}{dt} &= \alpha_n(V) (1 - n) - \beta_n(V)n = \frac{n^{\infty}(V) - n}{\tau_n(V)} \\
 \frac{1}{\gamma(T)} \frac{dm}{dt} &= \alpha_m(V) (1 - m) - \beta_m(V)m = \frac{m^{\infty}(V) - m}{\tau_m(V)} \\
 \frac{1}{\gamma(T)} \frac{dh}{dt} &= \alpha_h(V) (1 - h) - \beta_h(V)h = \frac{h^{\infty}(V) - h}{\tau_h(V)} \\
 \tau_n(V) &= \frac{1}{\alpha_n(V) + \beta_n(V)} \\
 n_{\infty}(V) &= \frac{\alpha_n(V)}{\alpha_n(V) + \beta_n(V)}
\end{align*}

en donde:
* $V$ la diferencia de potencial en la membrana
* $C_m = 1.0\ \mu\text{F/cm}^2$ la capacitancia de la membrana por unidad de área
* $g_{Na} = 120\ \text{mS/cm}^2$ la conductancia máxima por unidad de área para el sodio
* $g_{K} = 36\ \text{mS/cm}^2$ la conductancia máxima por unidad de área para el potasio
* $g_{L} = 0.3\ \text{mS/cm}^2$ la conductancia adicional, independiente del voltaje
* $E_{Na} = 55\ \text{mV}$ el voltaje de la batería en la compuerta de sodio
* $E_{K} = -72\ \text{mV}$ el voltaje de la batería en la compuerta de potasio
* $E_{L} = -49.387\ \text{mV}$ el voltaje filtrado
* $\gamma(T)$ es una constante de proporcionalidad temporal que depende de la temperatura $T$, a $6.3°$C vale $1$ [Cessac2009].

para el caso del calamar.  Los potenciales están medidos considerando que la membrana se encuentra en su potencial de reposo a los $V_r = -60\ \text{mV}$.  Las corrientes $I$ están medidas en $\mu$A/cm$^2$.

Las funciones de transición $\alpha(V)$ y $\beta(V)$, fueron determinadas experimentalmente.  Su forma general es:
\begin{align}
  \alpha(V) \text{ ó } \beta(V) = \frac{A + BV}{C + H e^{\left(\frac{V+D}{F}\right)}}
\end{align}
donde $V$ está medido con respecto al potencial de reposo $V_r$.

Los valores medidos por Hodkin y Huxley para la compuerta de **potasio (K)**, fueron:

\begin{align}
  \alpha_n(V) &= \frac{0.01(10 - V)}{e^{\left(\frac{10-V}{10}\right)} - 1}
\end{align}

\begin{align}
  \beta_n(V) = 0.125 e^{-\frac{V}{80}}
\end{align}

Observemos que cuando $V = 10$ la función $\alpha_n$ está indefinida.  Utilizando la regla de L'Hopital, se calcula el límite:
\begin{align}
  \alpha_n(10) = \frac{-0.01}{e^{\left(\frac{10-V}{10}\right)}(-0.1)} = \frac{-0.01}{-0.1} = 0.1
\end{align}

In [1]:
## Programa las función alfa_n como función del voltaje V.
## Necesitarás definir un caso especial cuando V - 10 < epsilon
## Utiliza esta celda para ver como se comporta tu función en esta vecindad
## y elegir un valor de epsilon adecuado

function alpha_n(V)
# Definimos nuestra epsilon adecuada
    epsilon = 0.001
    
# Realizamos el caso especial para V-10
# Utilizamos broadcasting con el operador .-
    if any(V .- 10 .< epsilon)  
        an = 0.01 * (10 .- V) ./ exp.(10 .- V ./ 10) .- 1
    else
# Valor de retorno predeterminado para el caso general
        an = zeros(size(V))  
    end
    
    return an
end

alpha_n (generic function with 1 method)

In [2]:
## Programa la funciones como función del voltaje V, para la compuerta de K
## Genera luego una gráfica de alfa_n y beta_n

function beta_n(V)
# En este caso beta no se indetermina, entonces la ecuación sería de la siguiente forma
    b_n=0.125*exp.(-V/80)
    return b_n
end


function plotAlpha()
    V = -150:0.1:150
#Aquí llamamos a nuestras funciones que definimos anteriormente
    alpha = alpha_n(V) 
    beta = beta_n(V) 
    plot(V,alpha,title="Tazas de transición",label="alfa(V)")
    plot!(V, beta, label="beta(V)")
end

plotAlpha (generic function with 1 method)

Las funciones para el **sodio (Na)** son:
\begin{align}
  \alpha_m(V) &= \frac{0.1(25 - V)}{e^{\left(\frac{25-V}{10}\right)} - 1}
\end{align}

\begin{align}
  \beta_m(V) &= 4 e^{-\frac{V}{18}}
\end{align}

\begin{align}
  \alpha_h(V) &= 0.07 e^{-\frac{V}{20}}
\end{align}

\begin{align}
  \beta_h(V) &= \frac{1}{e^{\left(\frac{30-V}{10}\right)} + 1}
\end{align}

Para $\alpha_m$ cuando $V = 25$, utilizamos la regla de L'Hopital para calcular el límite:
\begin{align}
  \alpha_m(25) = \frac{-0.1}{e^{\left(\frac{25-V}{10}\right)}(-0.1)} = \frac{-0.1}{-0.1} = 1
\end{align}

In [4]:
## Dadas alfa y beta, calcular ahora y graficar n y tau para ambos canales.

## Agrega las funciones correspondientes para la compuerta de sodio también,
## deduce sus definiciones a partir de las llamadas en la función para graficar
function alpha_m(V)
lst=[]
    for i in V
            if i == 25
                alpha_m= 1
                push!(lst, alpha_m)
            else 
                alpha_m= 0.1.*(25 .- i)./ (exp.((25 .- i) ./ 10) .- 1)
                push!(lst, alpha_m)
            end  
    end             
return lst
end


function beta_m(V)
    
# En este caso beta no se indetermina, entonces la ecuación sería de la siguiente forma
    b_m=4*exp.(-V/18)
    return b_m  
end


function alpha_h(V)

# En este caso beta no se indetermina, entonces la ecuación sería de la siguiente forma
    a_h=0.07*exp.(-V/20)
    return a_h
end


function beta_h(V)
    
    
       b_h = 1 ./ (exp.((30 .-V) ./ 10) .+ 1)
    
    
    return b_h
end

function beta_n(V)
    beta_n = 0.125.*(exp.(-V/80))
    return beta_n
end

function alpha_n(V)
   
   alpha = []
    for i in V
        if i == 10
            alpha_n = 0.1
            push!(alpha, alpha_n)
        else 
        alpha_n = (0.01*(10 .- i))./(exp.((10 .- i) ./ 10) .- 1)
                push!(alpha, alpha_n)
        end
    end
    
    return alpha
end
     
    
function tau_inf(alpha, beta)

    t_i = 1 ./ (alpha .+ beta)
      return t_i
end

function n_inf(alpha, beta)
    n_inf=  alpha ./ (alpha .+ beta)
end


function plotParameters()
    V = -100:0.1:100
    
    n_inf_n = n_inf(alpha_n(V), beta_n(V))
    tau_inf_n = tau_inf(alpha_n(V), beta_n(V))
    
    n_inf_m = n_inf(alpha_m(V), beta_m(V))
    tau_inf_m = tau_inf(alpha_m(V), beta_m(V))
    
    n_inf_h = n_inf(alpha_h(V), beta_h(V))
    tau_inf_h = tau_inf(alpha_h(V), beta_h(V))
    
    #p1 = plot([V, V, V])

    p1 = plot(V,[n_inf_n, n_inf_m, n_inf_h],label=["\\n_{infty}(V)","\\m_{infty}(V)","\\h_{infty}(V)"])
    p2 = plot(V,[tau_inf_n, n_inf_m, tau_inf_h],label=["\\tau_{n}(V)", "\\tau_{m}(V)", "\\tau_{h}(V)"])
    
    plot(p1,p2,layout=(1, 2),title=["Tazas de transición" "Constantes temporales"],xlabel=["V\\ realtivo \\a \\V_r (mV)","V \\realtivo \\a V_r\\ (mV)"], ylabel=["adimensional" "tau (ms)"],legend=true)
end

plotParameters (generic function with 1 method)

![tasas_constantes](figuras/tazas-y-constantes.jpg)

## Simulación con el método numérico de Euler

El método de Euler realiza una aproximación a la función por su tangente.  Dada una ecuación diferencial de la forma:

\begin{align}
  \frac{dy(t)}{dt} = f(y(t))
\end{align}

Partimos de un punto inicial $(t_0, y_0)$ y calculamos el valor de $y$ para el tiempo $t + \Delta t$ en dicho punto iterativamente como:

\begin{align}
  y(t + \Delta t) &= y(t) + \Delta t f(y(t))
\end{align}

El error aproximado por realizar esta aproximación es:
\begin{align}
  E =& \frac{1}{2} \frac{df(t)}{dt}(\Delta t)^2
\end{align}

Aplicando esta fórmula a la ecuación para los canales:

\begin{align}
 \frac{dn}{dt} &= \frac{n^{\infty}(V) - n}{\tau_n(V)}
\end{align}

Obtenemos:

\begin{align}
 n(t + \Delta t) =& n(t) + \Delta t \left[\frac{n^{\infty}(V)}{\tau_n} - \frac{n(t)}{\tau_n(V)} \right] \\
                 =& \left[ 1 - \frac{\Delta t}{\tau_n(V)} \right] n(t) + \frac{\Delta t}{\tau_n(V)}n^{\infty}(V)
\end{align}

Se obtienen fórmulas análogas para $m$ y $h$.  Todas juntas pueden ser escritas en forma matricial:

\begin{align}
  \begin{bmatrix}
  n(t + \Delta t) \\
  m(t + \Delta t) \\
  h(t + \Delta t)
  \end{bmatrix} =& 
  \begin{bmatrix}
  (1 - \Delta t/\tau_n(V)) & 0 & 0 \\
  0 & (1 - \Delta t/\tau_m(V)) & 0 \\
  0 & 0 & (1 - \Delta t/\tau_h(V)) \\
  \end{bmatrix}
  \begin{bmatrix}
  n(t) \\
  m(t) \\
  h(t)
  \end{bmatrix} +
  \begin{bmatrix}
  (\Delta t / \tau_n(V)) n^{\infty}(V) \\
  (\Delta t / \tau_m(V)) m^{\infty}(V) \\
  (\Delta t / \tau_h(V)) h^{\infty}(V)
  \end{bmatrix}
\end{align}

Brevemente:

\begin{align}
  \boldsymbol{\Pi}(t + \Delta t) =& \boldsymbol{A}_\pi \boldsymbol{\Pi}(t) + \boldsymbol{B}_\pi
\end{align}

Se debe realizar el mismo procedimiento con la ecuación diferencial para el voltaje.  Para simplificar la notación, introduzcamos:

\begin{align}
  G_{Na} = g_{Na}m^3h
\end{align}

\begin{align}
  G_{K} = g_K n^4
\end{align}

Entonces:
\begin{align}
 C_m \frac{dV}{dt} &= -g_{Na}m^3h(V-E_{Na}) - g_K n^4 (V - E_K) - g_L (V - E_L) + I_{ext} \\
 \frac{dV}{dt} &= -\frac{G_{Na}}{C_m}(V-E_{Na}) - \frac{G_K}{C_m} (V - E_K) - \frac{g_L}{C_m} (V - E_L) + \frac{1}{C_m}I_{ext}
\end{align}

Utilizando el método de Euler:

\begin{align}
 V(t + \Delta t) &= V(t) - \frac{\Delta t}{C_m} \left[ G_{Na}(V-E_{Na}) + G_K (V - E_K) + g_L (V - E_L) + I_{ext}(t) \right] \\
 V(t + \Delta t) &= V(t) - \Delta t \begin{bmatrix} \frac{G_{Na}}{C_m} & \frac{G_K}{C_m} & \frac{g_L}{C_m} \end{bmatrix}  \begin{bmatrix}
                 V(t)-E_{Na} \\
                 V(t)- E_K \\
                 V(t) - E_L
                \end{bmatrix} + \frac{\Delta t}{C_m}I_{ext}(t)
\end{align}

Para los valores por defecto propuestos en la celda siguiente, programar una simulación utilizando los resultados obtenidos con el método de Euler y produce la figura siguiente:

![simulacion](figuras/Simulacion.png)

In [5]:
## Implementa aquí el resultado del algoritmo de intregración numérica de Euler para calcular V(t).
# Asegúrate de que, al ejecutar la simulación con los parámetros por defecto, se reproduce la imagen de arriba.
# Después prueba con pulsos de t en 10->30, con corrientes de 10 y -10 microamperes
# ¿Qué observas?
# Inserta al final una celda con tus comentarios y resultados


# Primero tenemos que definir las variables que se van a usar en el siguiente código, 
# estás variables son proporcionadas por los datos de este mismo notebook, así que no son valores aleatorios.
const DeltaT = 0.01 # ms y es el intervalo de tiempo utilizado en la integración numérica.
const gL = 0.3 # mS/cm^2 y es la conductancia de fuga de la membrana.
const Cm = 1.0 # micro F/cm2 es la capacidad de la membrana.
const ENa = 55.0 # Voltaje de la bateria de la compuerta de Sodio (Na).
const EK = -72.0 # Voltaje de la bateria de la compuerta de Potasio (K).
const EL = -49.387 # Voltaje filtrado.
const maxT = 50.0 # ms que es el tiempo máximo de simulación.
const v0 = 0.0 # mV que es voltaje inicial.


# Empezando con los códigos dados por el notebook:

# Se define la primera función "simulaHodkinHuxley(v0, t0, tfin, current)" donde vamos a utilizar las funciones 
# definidas en los códigos de arriba:

function simulaHodkinHuxley(v0, t0, tfin, current)
    
n_inf_n(V) = n_inf(alpha_n(V), beta_n(V))
n_inf_m(V) = n_inf(alpha_m(V), beta_m(V))
n_inf_h(V) = n_inf(alpha_h(V), beta_h(V))
    
function Iext(t)
        
    """ Devuelve el valor de la corriente aplicada a la membrana, al tiempo t, en microampers."""
        
    if (t > t0) & (t < tfin)
        return current # micro A/cm2
    else 
        return 0
    end
end

# Según las ecuaciones anteriormente mostradas, definimos una función que va a utilizar ecuaciones diferenciales, 
# por lo cual, se muestran las respectivas derivadas.
    
function HodkinHuxley(x, y, z)
        
    n, m, h, V = x 
        
    V_diff = V - v0
        
    dn = alpha_n(V_diff) * (1 - n) - beta_n(V_diff) * n
    dm = alpha_m(V_diff) * (1 - m) - beta_m(V_diff) * m
    dh = alpha_h(V_diff) * (1 - h) - beta_h(V_diff) * h
    
# Cálculo de las conductancias de potasio y sodio respectivamente.
    G_K= gK * (n ^ 4.0) 
    G_Na= gNa * (m ^ 3.0) * h 

    dV = (Iext(t) + (ENa - V) * G_Na + (EK - V) * G_K + (EL - V) * gL) / Cm 
            
    [dn; dm; dh; dV]
end
    
x0 = [n_inf_n(0); n_inf_m(0); n_inf_h(0); 0] 
ts = (0.0:50.0) 
pro = ODEProblem(HodkinHuxley, x0, ts) 
sol = solve(pro, saveat = 0.01);
G_K_s = gK * sol[1,:].^ 4
G_Na_s = gNa * (sol[2,:].^ 3.0) .* sol[3,:]

#Ahora, para poder gráficar, se van a definir las siguientes cosas:
    
function VoltajeM()
        
plot(sol.t, G_K_s, title ="Conductancia de los canales", label = "G_K", xlabel = "t (ms)", ylabel = "Conductancia (ms/cm^2)")
plot!(sol.t, G_Na_s, label = "G_Na")
        
end

# La indicación "zeros(length(sol.t)" crea y devuelve un vector de soluciones.
    
function ConstT()
    
tau_h(V) = tau_inf(alpha_h(V), beta_h(V))

function tau_h_s()
            
    t_h_s = zeros(length(sol.t))
            
    for i in 1:(length(sol.t))
        t_h_s[i] = tau_h(sol[4,:][i])
    end
    return t_h_s
end

tau_m(V)=tau_inf(alpha_m(V), beta_m(V))
        
function tau_m_s()
            
    t_m_s = zeros(length(sol.t))
            
    for i in 1:(length(sol.t))
        t_m_s[i] = tau_m(sol[4,:][i])
    end
    return t_m_s
end

tau_n(V)=tau_inf(alpha_n(V), beta_n(V))

function tau_n_s()
            
    t_n_s = zeros(length(sol.t))
            
    for i in 1:(length(sol.t))
        t_n_s[i] = tau_n(sol[4,:][i])
    end
    return t_n_s
end
#Llamamos las gráficas, para esto es necesario tener la paqueteria "PLOT" de "PLOTS.JL".
        
plot(sol.t,tau_h_s(), title = "Constantes temporales", xlabel = "t (ms)", ylabel = "tau (ms)", label = "tau_h")
plot!(sol.t,tau_m_s(), label = "tau_m")
plot!(sol.t,tau_n_s(), label = "tau_n")
    
end

p1 = plot(sol.t, sol[4,:], legend = false, ylabel = "Voltaje relativo en la membrana (mV)", xlabel = "t (ms)", title = "Voltaje en la membrana")
p2 = VoltajeM()
p3 = plot(sol.t, sol[1:3,:]', label = ["n" "m" "h"], legend =:topright, ylabel = "adimensional", xlabel = "t (ms)", title = "Tazas de transicion")
p4 = ConstT()

l = (2, 2)

#Con la siguiente orden podemos ver varios gráficos como subplots de otro más amplio y compararlos:
plot(p1, p2, p3, p4, layout = l, size = (1000,1000))    
end

simulaHodkinHuxley (generic function with 1 method)

In [ ]:
# Implementa aquí el resultado del algoritmo de intregración numérica de Euler para calcular V(t).
# Asegúrate de que, al ejecutar la simulación con los parámetros por defecto, se reproduce la imagen de arriba.
# Después prueba con pulsos de t en 10->30, con corrientes de 10 y -10 microamperes
# ¿Qué observas?
# Inserta al final una celda con tus comentarios y resultados

function EulerHodkinHuxley(parametros)
    """Devuelve la solucion a las ecuaciones diferenciales del modelo de Hodgkin
    y Huxley utilizando el metodo de Euler
    """
end
  

function plotSimulation(resultado)
    tiempo = resultado["T"]
    
    p1 = plot(tiempo, resultado["V"])
    p2 = plot(tiempo, [resultado["GK"], resultado["GNa"]], label=[L"G_{K}" L"G_{Na}"])
    p3 = plot(tiempo, [resultado["n"], resultado["m"], resultado["h"]], label=[L"n" L"m" L"h"])
    p4 = plot(tiempo, [resultado["tauN"], resultado["tauM"], resultado["tauH"]], label=[L"\tau_n" L"\tau_m" L"\tau_h"])

    plot(
        p1,
        p2,
        p3,
        p4,
        layout=(2, 2),
        title=["Voltaje en la membrana" "Conductancia de los canales" "Tazas de transición" "Constantes temporales"],
        xlabel="t (ms)",
        ylabel=["Voltaje relativo en la membrana (mV)" L"Conductancia (mS/cm^2)" "adimensional" L"\tau (ms)"],
        legend=true
    )
end
    

function simulaHodkinHuxley(V0, t0, tfin, current)
    iext_func = makeIext(t0, tfin, current)
    parametros["V0"] = V0
    resultado = EulerHodkinHuxley(parametros)
    plotSimulation(resultado)
end


V0 = 0
t0 = 10
tfin = 16 # seleccionar de: -90 a 120
current = 2.4 # seleccionar de: -10 a 10

simulaHodkinHuxley(V0, t0, tfin, current)

# Comentarios y resultados
Escribe aquí tus resultados